In [10]:
import os
import pandas as pd
import numpy as np
import mypytable as mypy
from sklearn.model_selection import train_test_split
import tensorflow as tf

stockfiles= os.listdir("/home/cbarker4/Documents/DataScience/StockTrader/Data")
print(stockfiles)

['AADI.csv', 'AAPL.csv', 'ABCL.csv', 'ABIO.csv', 'AACG.csv', 'AAL.csv', 'ABEO.csv', 'AAON.csv', 'AACI.csv', 'AAME.csv', 'ABCM.csv', 'ABCB.csv', 'AACIW.csv', 'ABNB.csv', 'AAOI.csv']


In [11]:


X = []
Y=[]
Xlast=[]
for val in stockfiles:
    print(val)

    mt = mypy.MyPyTable()
    mt.load_from_file("/home/cbarker4/Documents/DataScience/StockTrader/Data/"+stockfiles[0])
    mt.drop_column('v')
    mt.drop_column('t')
    mt.drop_column('s')
    i = 100
    while i <  + len(mt.data)-1:
        table = mt.create_sub_table(i-100,i)

        table.drop_column('v')
        table.drop_column('t')
        X.append(table.data)
        Xlast.append(table.data[-1])
        Y.append(mt.get_row(i+1)[0])
        i+=1
    
X = np.array(X)
# print(Y)
Y = np.array(Y)
# print(Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25 ,random_state=0)


callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor = 'loss',
        patience = 7,
        verbose = 1,
        min_delta = 0,
        mode = 'min',
        baseline = None,
        restore_best_weights = True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join("/home/cbarker4/Documents/DataScience/StockTrader/Model", 'ckpt', "{epoch:02d}-{val_loss:.2f}.hdf5"),
        monitor = 'loss',
        verbose = 1,
        save_best_only = True,
        save_weights_only = False,
        mode = 'min',
        save_freq = 'epoch',
        options = None,
        initial_value_threshold = None
    )   
]

AADI.csv
AAPL.csv
ABCL.csv
ABIO.csv
AACG.csv
AAL.csv
ABEO.csv
AAON.csv
AACI.csv
AAME.csv
ABCM.csv
ABCB.csv
AACIW.csv
ABNB.csv
AAOI.csv


In [12]:

loss_fun=["mean_absolute_error"]

for funct in loss_fun:

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(units=64,
                                return_sequences=True,
                                input_shape=(X_train.shape[1], 5)))
    model.add(tf.keras.layers.LSTM(units=64))
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    # model.summary


    model.compile(optimizer='adam',loss = 'mean_absolute_error')#,metrics=accuracy)
    history = model.fit(X_train,Y_train,epochs = 500, 
                                        validation_split = 0.2, 
                                        callbacks = callbacks)


    y_pred = model.predict(X_test)
    Xlast=[]
    correct = 0
    for val in X_test:
        Xlast.append(val[-1][0])


    for i,val in enumerate(Y_test):
        
        if Xlast[i] > val:
            if Xlast[i]>y_pred[i]:
                correct+=1
        else:
            if Xlast[i]<y_pred[i]:
                correct+=1 

    print('Correct predictions: ', correct)
    print('Incorrect predictions: ', len(y_pred) - correct)
    print('Accuracy: ', correct / len(y_pred))
    

Epoch 1/500


2023-04-07 14:08:40.098198: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:433] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-04-07 14:08:40.098232: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at cudnn_rnn_ops.cc:1554 : UNKNOWN: Fail to find the dnn implementation.


UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_4/lstm_8/PartitionedCall]] [Op:__inference_train_function_21957]